In [5]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.utils.coco as fouc
import pandas as pd

### 1 Dataset generation

#### 1.1 Downloading OpenImages dataset

In [ ]:
classes = ["Human hand", "Human head", "Person", "Hat", "Dress", "Human hair", "Human eye", "Human mouth", "Tree", "Animal", "Building"]

dataset = foz.load_zoo_dataset(
    "open-images-v7",
    split="validation",
    label_types=["detections", "classifications"],
    classes=classes,
    dataset_name="dataset",
    load_hierarchy=True,
    dataset_dir="/Volumes/Samsung_USB/open-images",
)

session = fo.launch_app(dataset)

#### 1.2 Exporting OpenImages to COCO format

In [ ]:
dataset.export(
   export_dir="/Volumes/Samsung_USB/coco-dataset",
   dataset_type = fo.types.COCODetectionDataset,
   label_field= "detections",
)

### 2 Dataset evaluation

#### 2.1 Downloading COCO dataset
data_path corresponds to the directory where images of paintings are stored, labels_path corresponds to json annotation file with ground truth bounding boxes.

In [ ]:
coco_dataset = fo.Dataset.from_dir(
    dataset_type=fo.types.COCODetectionDataset,
    data_path="/Volumes/SAMSUNG_USB/test-images", 
    labels_path="/Volumes/SAMSUNG_USB/annotations/test-ann-27-06.json", 
    include_id=True,
)

# Verify that the class list for the dataset was imported
print(coco_dataset.default_classes) # ['0', 'Animal', 'Building', 'Dress'...]
print(coco_dataset)

In [7]:
df = pd.read_json("/Volumes/SAMSUNG_USB/annotations/predictions.json")
predictions = df.to_json(orient='records')

# Add COCO predictions to `predictions` field of dataset
classes = coco_dataset.default_classes
fouc.add_coco_labels(coco_dataset, "predictions", predictions, classes)

# Verify that predictions were added to nineteen painting images
print(coco_dataset.count("predictions"))  # 19

19


#### 2.3 Displaying ground truth and predicted bounding boxes

In [ ]:
session = fo.launch_app(coco_dataset)